In [125]:
# from pyspark import SparkConf, SparkContext
# config= SparkConf().setMaster("local[*]").setAppName("python optimization")
# sc= SparkContext(conf=config)
# # sc.stop()

In [99]:
sc

AttributeError: 'NoneType' object has no attribute 'sc'

<SparkContext master=local[*] appName=PySparkShell>

In [57]:
spark

In [3]:
hr_employee = spark.read.csv("file:///home/hadoop/Downloads/HR_Employee.csv", inferSchema = True, header = True)

In [4]:
hr_employee.printSchema()

root
 |-- EmployeeID: integer (nullable = true)
 |-- Department: string (nullable = true)
 |-- JobRole: string (nullable = true)
 |-- Attrition: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- EducationField: string (nullable = true)
 |-- BusinessTravel: string (nullable = true)
 |-- JobInvolvement: string (nullable = true)
 |-- JobLevel: integer (nullable = true)
 |-- JobSatisfaction: string (nullable = true)
 |-- Hourlyrate: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Salaryhike: integer (nullable = true)
 |-- OverTime: string (nullable = true)
 |-- Workex: integer (nullable = true)
 |-- YearsSinceLastPromotion: integer (nullable = true)
 |-- EmpSatisfaction: string (nullable = true)
 |-- TrainingTimesLastYear: integer (nullable = true)
 |-- WorkLifeBalance: string (nullable = true)
 |-- Performance_Rating: string (nul

#### 1. Big DataFile Types
- parquet FileFormat - Records are stored Columnar format, this file format compresses dataset of .csv of structured format into parquet format, parquet format is good for query type of response.
- There are other file formats as well like - AVRO, ORC formats

In [6]:
hr_employee.rdd.getNumPartitions()

1

In [7]:
hr_employee.write.parquet('file:///home/hadoop/Downloads/HR_Parquet')

In [8]:
hr_employee.write.orc('/HR_Orc')

In [9]:
spark.read.orc('/HR_Orc').show(10)

+----------+--------------------+--------------------+---------+------+---+-------------+-------------+--------------+-----------------+--------------+--------+---------------+----------+------+----------+--------+------+-----------------------+---------------+---------------------+---------------+------------------+
|EmployeeID|          Department|             JobRole|Attrition|Gender|Age|MaritalStatus|    Education|EducationField|   BusinessTravel|JobInvolvement|JobLevel|JobSatisfaction|Hourlyrate|Income|Salaryhike|OverTime|Workex|YearsSinceLastPromotion|EmpSatisfaction|TrainingTimesLastYear|WorkLifeBalance|Performance_Rating|
+----------+--------------------+--------------------+---------+------+---+-------------+-------------+--------------+-----------------+--------------+--------+---------------+----------+------+----------+--------+------+-----------------------+---------------+---------------------+---------------+------------------+
|         1|               Sales|     Sales

#### Optmization Techniques
- Optimizing Spark jobs can significantly improve performance of spark running queries, spark jobs.
2. Paritioning
- Partitioning divides data into smaller chunks, which can be processed parallely.

In [10]:
hr_employee.rdd.getNumPartitions()

1

In [11]:
partitioned_df = hr_employee.repartition(3)

In [12]:
partitioned_df.write.parquet("/HR_partition")

#### 3. Caching & Persistance
- Managing Different level of storage

In [13]:
# In-Memory Cache storage
hr_employee.cache()

DataFrame[EmployeeID: int, Department: string, JobRole: string, Attrition: string, Gender: string, Age: int, MaritalStatus: string, Education: string, EducationField: string, BusinessTravel: string, JobInvolvement: string, JobLevel: int, JobSatisfaction: string, Hourlyrate: int, Income: int, Salaryhike: int, OverTime: string, Workex: int, YearsSinceLastPromotion: int, EmpSatisfaction: string, TrainingTimesLastYear: int, WorkLifeBalance: string, Performance_Rating: string]

In [14]:
# Persistance of DataFrame with specific type of storage options like - memory only
# memory_ser, memory and disk

from pyspark import StorageLevel
hr_employee1 = hr_employee.persist(StorageLevel.MEMORY_AND_DISK) #Temporary storage

In [15]:
hr_employee2 = hr_employee.persist(StorageLevel.MEMORY_ONLY_SER) #Temporary storage

#### 4. Serialization
- Efficient serialization reduces time to read write data and transfer it over network.
- Kyro Serialization is the popular serialization method for better performance over default java serialization.

### a. Java Serialization:
It is default serialization method, Its easy to use but drawback is it will slow down the read, write process

In [16]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [17]:
spark = SparkSession.builder.appName("Java serialization").getOrCreate()

In [18]:
spark.stop()

In [19]:
spark = SparkSession.builder.appName("pyspark serialization")\
.config("spark.serializer", "org.apache.spark.serializer.JavaSerializer").getOrCreate()

In [21]:
spark

b) Kyro Serialization: Faster more compact than JavaSerialization

In [22]:
spark = SparkSession.builder\
.config("spark.serializer", "org.apache.spark.serializer.KyroSerializer")\
.config("spark.kyro.registrationRequired", "true")\
.config("spark.kyro.classesToRegister","org.apache.spark.example.Person")\
.appName("Kyro Serialization").getOrCreate()

#### 5. Broadcast Joins
- Broadcasting small datasets, improve join performance

In [37]:
small_df = spark.read.csv("file:///home/hadoop/Downloads/airports.csv", header= True, inferSchema=False)
df = spark.read.csv("file:///home/hadoop/Downloads/raw_flight_data1.csv", header= True, inferSchema=False)

In [38]:
from pyspark.sql.functions import broadcast

In [39]:
broadcast_df = broadcast(small_df)

In [40]:
broadcast_df = broadcast_df.cache()
df = df.cache()

In [42]:
broadcast_df.show(10)

+----------+-----------+-----+--------------------+
|airport_id|       city|state|                name|
+----------+-----------+-----+--------------------+
|     10165|Adak Island|   AK|                Adak|
|     10299|  Anchorage|   AK|Ted Stevens Ancho...|
|     10304|      Aniak|   AK|       Aniak Airport|
|     10754|     Barrow|   AK|Wiley Post/Will R...|
|     10551|     Bethel|   AK|      Bethel Airport|
|     10926|    Cordova|   AK|Merle K Mudhole S...|
|     14709|  Deadhorse|   AK|   Deadhorse Airport|
|     11336| Dillingham|   AK|  Dillingham Airport|
|     11630|  Fairbanks|   AK|Fairbanks Interna...|
|     11997|   Gustavus|   AK|    Gustavus Airport|
+----------+-----------+-----+--------------------+
only showing top 10 rows



In [44]:
airport_df = df.join(broadcast_df, df.OriginAirportID == broadcast_df.airport_id)

In [45]:
airport_df.show(2)

+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|          city|state|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|        19|        5|     DL|          11433|        13303|      -3|       1|     11433|       Detroit|   MI|Detroit Metro Way...|
|        19|        5|     DL|          14869|        12478|       0|      -8|     14869|Salt Lake City|   UT|Salt Lake City In...|
+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
only showing top 2 rows



#### 6. Level of parallelism

In [46]:
# Adjust level of parallelism based on your cluster size
spark.conf.set("spark.default.parallelism",100)

#### 7. Avoid GroupByKey
- Use ReduceByKey or aggregateByKey() instead of GroupByKey() to reduce number of shuffling

In [58]:
rdd = spark.sparkContext.parallelize([('dosa',2),('coffee',3), ('idly',3), ('vada',5), ('rice',1), ('coffee',2), ('vada',1)])
rdd.groupByKey().mapValues(sum).collect()

[('dosa', 2), ('coffee', 5), ('idly', 3), ('vada', 6), ('rice', 1)]

In [54]:
rdd.reduceByKey(lambda x, y : x+y).collect()

[('dosa', 2), ('coffee', 5), ('idly', 3), ('vada', 6), ('rice', 1)]

In [67]:
from pyspark.sql.functions import *
df = spark.createDataFrame([('dosa',2),('coffee',3), ('idly',3), ('vada',5), ('rice',1), ('coffee',2), ('vada',1)], ['order','value'])

x = df.groupBy("order").agg(sum("value").alias("Total Value"))
x.show()

+------+-----------+
| order|Total Value|
+------+-----------+
|  vada|          6|
|  dosa|          2|
|  idly|          3|
|  rice|          1|
|coffee|          5|
+------+-----------+



In [68]:
df.rdd.reduceByKey(lambda x, y : x+y).collect()

[('dosa', 2), ('coffee', 5), ('idly', 3), ('vada', 6), ('rice', 1)]

#### 8. Reduce shuffle
- Reduce the no.of shuffles by optimizing transformations
- use reduceByKey over groupByKey()
- use Map() and reduce() over groupBy()

#### 9. Repartition() and Coalesce()

#### 10. Accumulators
- use Accumulators for optimizing aggregate information like count(), sum(). Across all executors parallely executing tasks in multiple worker nodes
- Accumulator in spark are variables that can be added through cummulative operations.

In [118]:
acc = spark.sparkContext.accumulator(0)

In [119]:
type(acc)

pyspark.accumulators.Accumulator

In [120]:
rdd = spark.sparkContext.parallelize([1,2,3,4,5,6,7,8,9])


In [121]:
# python udf
def add(x):
    acc.add(x) 

In [122]:
rdd.foreach(add)

In [123]:
print(acc.value)

45


In [112]:
def counter(x):
    acc.add(1)
    return x

In [113]:
rdd.map(counter).count()

9

#### 11. Bucketing
- Use Bucketing to create buckets of large datasets for efficient query and joins